<a href="https://colab.research.google.com/github/RamyaAL3362/skilldevelopment/blob/main/multimodal_RAG_with_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install all necessary packages
!pip install langchain sentence-transformers faiss-cpu pytesseract pillow transformers gradio
!pip install -U langchain-community

# Import necessary libraries
import pytesseract
from PIL import Image
import faiss
import gradio as gr
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import torch

# 🔍 Step 1: Extract text from image using OCR
def extract_text_from_image(image: Image.Image) -> str:
    text = pytesseract.image_to_string(image)
    return text

# 🧠 Step 2: Initialize all models (embedding + QA model)
def setup_models():
    # Embedding model from Sentence Transformers (No API key needed)
    embedding_model_name = "all-MiniLM-L6-v2"
    embedder = HuggingFaceEmbeddings(model_name=embedding_model_name)

    # QA Model (Local Hugging Face model)
    hf_qa_pipeline = pipeline(
        "question-answering",
        model="distilbert-base-uncased-distilled-squad",
        tokenizer="distilbert-base-uncased",
        device=0 if torch.cuda.is_available() else -1
    )
    llm = HuggingFacePipeline(pipeline=hf_qa_pipeline)

    return embedder, llm

# 📦 Step 3: Create FAISS Vector Store from OCR’d text
def create_vectorstore_from_text(text: str, embedder):
    # Split text into chunks
    splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_text(text)
    docs = [Document(page_content=chunk) for chunk in chunks]

    # Create FAISS vector store
    vectorstore = FAISS.from_documents(docs, embedder)
    return vectorstore

# 🧩 Step 4: Complete RAG Chain
def run_rag_pipeline(image, question):
    # OCR Text
    extracted_text = extract_text_from_image(image)

    if not extracted_text.strip():
        return "❌ No text detected in the image."

    # Setup models
    embedder, llm = setup_models()

    # Vector store
    vs = create_vectorstore_from_text(extracted_text, embedder)

    # LangChain RetrievalQA
    rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vs.as_retriever())

    # Run query
    response = rag_chain.run(question)

    return f"**Answer:** {response}"

# 🖼️ + 💬 Gradio UI
image_input = gr.Image(type="pil", label="Upload Image with Text")
question_input = gr.Textbox(label="Ask a Question")
output = gr.Markdown()

gr.Interface(
    fn=run_rag_pipeline,
    inputs=[image_input, question_input],
    outputs=output,
    title="🔍 Multimodal RAG with LangChain (No API)",
    description="Upload an image containing text and ask a question. Uses OCR + LangChain + FAISS. No API keys needed!"
).launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.